In [65]:
#importing Library
import pymongo
from pymongo import MongoClient
import pandas as pd
import numpy as np
from datetime import datetime , timedelta

In [66]:
#importing database from MongoDB using pymongo
connection = MongoClient('localhost',27017)
database = connection.Customer

In [67]:
# We have 3 database in MongoDB
# Customer Database : information about customer
customer_data = database.customer_db
customer_df = pd.DataFrame(list(customer_data.find()))

#farm database : information about farm 
farm_data = database.farm_db
farm_df = pd.DataFrame(list(farm_data.find()))

#device databse : information about device
device_data = database.Device_db
device_df = pd.DataFrame(list(device_data.find()))

In [68]:
# using merge() we are joining farm database to device database using farm_id as primary key
new_df = pd.merge(farm_df,device_df,how="inner",on="farm_id")

# dropping unnecessary attributes
new_df.drop(['lat','_id_x','customer_id','long','name','_id_y','device_id_x'],axis=1,inplace=True)
new_df.head(10)

,crop,farm_id,device_id_y,TC,HUM,datetime,LW,PLV2
0,tomato,613f0a6528ae98f927f4764f,7HGRXY,20,70,2021-07-30 03:48:29.044,0,0
1,tomato,613f0a6528ae98f927f4764f,7HGRXY,28,75,2021-07-30 10:48:29.044,0,0
2,tomato,613f0a6528ae98f927f4764f,7HGRXY,32,80,2021-07-30 11:48:29.044,1,0
3,tomato,613f0a6528ae98f927f4764f,7HGRXY,20,73,2021-07-30 12:48:29.044,1,0
4,tomato,613f0a6528ae98f927f4764f,7HGRXY,22,74,2021-07-30 00:00:00.000,1,0


In [74]:
# Creating a datetime object and date_list contain every unique date
new_df['datetime'] = pd.to_datetime(new_df['datetime'])
date_list = pd.DataFrame({'date':pd.to_datetime(new_df['datetime']).dt.date.unique()})

# converting date_list dataframe into datetime object
date_list["date"] = pd.to_datetime(date_list["date"])

# Creating get_data function 
def get_data(crop_name,start_date,end_date,sensor_value):
    # Creating dataframe contain 24 hour record
    oneday_df = new_df[(new_df["datetime"]>=start_date) & (new_df["datetime"]<end_date)]
    hourly_data = oneday_df.groupby(pd.Grouper(key="datetime",freq="1H")).mean()
    get_df = pd.concat([hourly_data,crop_name],axis=1)
    return get_df
    
    
for i in date_list["date"]:
    start_date = i
    if start_date == date_list["date"].iloc[-1]:
        break
    end_date = i + timedelta(1)
    sensor_value = new_df[["TC","HUM","LW","PLV2"]]
    crop_name = new_df["crop"]
    print(get_data(crop_name,start_date,end_date,sensor_value))


In [70]:
# This NaN value occurs because in MongoDB I inserted few sample to check whether this function works or not